# AgCl as first test case

First test runs comparing tableau based solver (numerical or analytical gradients and logX versus X solver) and PHREEQC.

In [511]:
% setup environment
% set path to equilibrium solver
txt=pwd;
n=length(txt);
rootpath=txt; rootpath=txt(1:n-11);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% clear data and number format
clear; format short e
% turn off warnings
warning off
%graphics_toolkit ("notebook"); 
%graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")

In [516]:
% MODEL USING TABLEAU (different gradients or log versus linear variables)
%graphics_toolkit("notebook")
%model with just inorganic speciation
%geochemical model
% input the constants and solution chemistry

flag1=1; %flag1 for concentration based search vector (1) or for log based search vector (2).
flag2=1; %flag2 for numerical derivatives (2) or analtyical derivatives (1)
%database=[]; 
flag3=0; % flag3 1 for show warnings. 0 no warnings
flag4=0; %if=1 solve tableau one line at a time to get a good initial guess (no solids). really challenging probs only
flag5=0; %if=1 use stored initial guess for including solids

AgT=1e-2; ClT=1e-3; pH=7; 

TOTALS=[AgT ClT];
TOTALS(TOTALS==0)=1e-16; % get rid of zero values, b/c div by zero error
%TOTALS=[inorganicTOTALS]; %TOTALS=inorganicTOTALS;
pH=pH; pe=20.75-pH; 

tic;
[Ag,Cl,AgCls,MASSERR]=AgCltableau(pH,pe,TOTALS',flag1,flag2,flag3,flag4,flag5);
tableautime=toc;

Agmasserror=MASSERR(1); Clmasserror=MASSERR(2);

X no ITER EXEED no solid logX


In [517]:
% MODEL USING PHREEQC

minerals=[{'AgCls'}]; totalvector=[AgT; ClT; ClT]; totalnames=[{'Ag'}; {'Cl'}; {'Na'}]; speciesexport=[{'Ag+'}; {'Cl-'};{'AgCl'};{'AgCl2-'};{'AgCl3-2'};{'AgCl4-3'}];
%database=['llnl_nosolubleAgCl.dat']; 
acid=['NaOH']; show=0; T=25;
%database=['AgCl.dat']; 
database=['complexAgCl.dat']; 

tic
[solutionspeciesconcs, speciesnames, solidconcs, solidnames]=runPHREEQCv2noHA(T,pH,pe,totalnames,totalvector,minerals,speciesexport,database,show,acid);
PHREEQCtime=toc;

AgPHREEQC=solutionspeciesconcs(1); ClPHREEQC=solutionspeciesconcs(2); AgClsPHREEQC=solidconcs(1);
AgClPHREEQC=solutionspeciesconcs(3); AgCl2PHREEQC=solutionspeciesconcs(4); AgCl3PHREEQC=solutionspeciesconcs(5); AgCl4PHREEQC=solutionspeciesconcs(6);

AgmasserrorPHREEQC=AgT-AgPHREEQC-AgClsPHREEQC-AgClPHREEQC-AgCl2PHREEQC-AgCl3PHREEQC-AgCl4PHREEQC; 
ClmasserrorPHREEQC=ClT-ClPHREEQC-AgClsPHREEQC-AgClPHREEQC-2*AgCl2PHREEQC-3*AgCl3PHREEQC-3*AgCl4PHREEQC;

Input file: runphreeqc.txt

Output file: out.txt

Database file: complexAgCl.dat


Initializing...                                                                 

End of Run after 0.00373 Seconds.


In [518]:
% exact model

Ksp=10^-9.7453;

% check SI

IAP=AgT*ClT;

Agexact=AgT; Clexact=ClT; AgClsexact=0;

if IAP>=Ksp
a=1; b=ClT-AgT; c=-Ksp;
t=roots([a b c]); t=t(t>0); t=real(t(imag(t)==0));
Agexact=t; AgClsexact=AgT-Ag; Clexact=ClT-AgClsexact;
end


In [515]:
% compare models

Agcompare=[Ag AgPHREEQC Agexact]
Clcompare=[Cl ClPHREEQC Clexact]
AgClcompare=[AgCls AgClsPHREEQC AgClsexact]
Agmasserrorcompare=[Agmasserror AgmasserrorPHREEQC]
Clmasserrorcompare=[Clmasserror ClmasserrorPHREEQC]
timecompare=[tableautime PHREEQCtime]

Agcompare =

   9.0000e-03   9.0000e-03   9.0000e-03

Clcompare =

   1.9974e-08   2.3243e-08   1.9974e-08

AgClcompare =

   9.9998e-04   9.9962e-04   9.9998e-04

Agmasserrorcompare =

   1.7347e-14   7.4230e-15

Clmasserrorcompare =

            0   7.8106e-18

timecompare =

   7.8813e-02   2.1303e-02

